<br>

<div align=center><font color=maroon size=6><b>The Sequential model</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 Core: <a href="https://www.tensorflow.org/guide" style="text-decoration:none;">TensorFlow Guide</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Guide > <a href="https://www.tensorflow.org/guide/keras/sequential_model" style="text-decoration:none;">The Sequential model</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/docs/blob/snapshot-keras/site/en/guide/keras/sequential_model.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
tf.__version__

'2.8.0'

<br>

## When to use a Sequential model

A `Sequential` model is appropriate for **a plain stack of layers**
where each layer has **exactly one input tensor and one output tensor**.

Schematically, the following `Sequential` model:

In [3]:
# Define Sequential model with 3 layers
model = keras.Sequential([layers.Dense(2, activation="relu", name="layer1"),
                          layers.Dense(3, activation="relu", name="layer2"),
                          layers.Dense(4, name="layer3"),
                         ])

# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

In [4]:
print(x)
print(y)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(3, 4), dtype=float32)


<br>

<font size=3 color=maroon>is equivalent to this function:</font>

In [5]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

<br>

<font size=3 color=maroon>A Sequential model is **not appropriate** when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)</font>

<br>
<br>
<br>

## Creating a Sequential model

You can create a Sequential model by passing a list of layers to the Sequential
constructor:

In [6]:
model = keras.Sequential([layers.Dense(2, activation="relu"),
                          layers.Dense(3, activation="relu"),
                          layers.Dense(4),
                         ])

Its layers are accessible via the `layers` attribute:

In [7]:
model.layers

<br>

You can also create a Sequential model incrementally via the `add()` method:

In [8]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

In [9]:
print(len(model.layers))

3


<br>

<font size=3 color=maroon>**Note that**</font> there's also a corresponding `pop()` method to remove layers:<br>
<font size=3 color=maroon>a Sequential model behaves very much like a list of layers.</font>

In [10]:
model.pop()
print(len(model.layers))  # 2

2


<br>

<font size=3 color=maroon>**Also note that**</font> the Sequential constructor accepts a `name` argument, just like any layer or model in Keras. <font size=3 color=maroon>This is useful to annotate TensorBoard graphs with semantically meaningful names.</font>

In [11]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

<br>
<br>
<br>

## Specifying the input shape in advance

Generally, all layers in Keras need to know the shape of their inputs
in order to be able to create their weights. So when you create a layer like
this, initially, it has no weights:

In [12]:
layer = layers.Dense(3)
layer.weights  # Empty

[]

<br>

It creates its weights <font size=3 color=maroon>the first time it is called on an input, since the shape of the weights depends on the shape of the inputs:</font>

In [13]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights      # Now it has weights, of shape (4, 3) and (3,)

[<tf.Variable 'dense_6/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.5997952 , -0.7993725 , -0.312186  ],
        [ 0.47801363, -0.29375046, -0.3105694 ],
        [-0.69011617,  0.8706579 , -0.6269699 ],
        [ 0.11025929, -0.48978028,  0.8577424 ]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

<br>

Naturally, this also applies to Sequential models. <font size=3 color=maroon>When you instantiate a Sequential model without an input shape, it isn't "built": it has no weights (and calling `model.weights` results in an error stating just this). The weights are created when the model first sees some input data:</font>

In [14]:
model = keras.Sequential([layers.Dense(2, activation="relu"),
                          layers.Dense(3, activation="relu"),
                          layers.Dense(4),])  # No weights at this stage!

In [15]:
# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Number of weights after calling the model: 6


<br>

Once a model is "built", you can call its `summary()` method to display its contents:`

In [16]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (1, 2)                    10        
                                                                 
 dense_8 (Dense)             (1, 3)                    9         
                                                                 
 dense_9 (Dense)             (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


<br>

However, it can be very useful when building a Sequential model incrementally to be able to display the summary of the model so far, including the current output shape. <font size=3 color=maroon>In this case, you should start your model by passing an `Input` object to your model, so that it knows its input shape from the start:</font>

In [17]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation='relu'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


<br>

<font size=3 color=maroon>**Note that** the `Input` object is not displayed as part of `model.layers`, since it isn't a layer:</font>

In [18]:
model.layers

<br>

<font size=3 color=maroon>A simple alternative is to just pass an `input_shape` argument to your first layer:</font>

In [19]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


<br>
<br>
<br>

## A common debugging workflow: `add()` + `summary()`
<br>

<font size=3 color=maroon>When building a new Sequential architecture, it's useful to incrementally stack layers with `add()` and frequently print model summaries.</font> 

For instance, this enables you to monitor how a stack of `Conv2D` and `MaxPooling2D` layers is downsampling image feature maps:

In [20]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()
print('\n\n')



# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 40, 40, 32)       0         
 )                                                               
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________



Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                         

<br>

In [21]:
# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Very practical, right?

<br>

## What to do once you have a model

Once your model architecture is ready, you will want to:

- Train your model, evaluate it, and run inference. See our [guide to training & evaluation with the built-in loops](https://www.tensorflow.org/guide/keras/train_and_evaluate/)
- Save your model to disk and restore it. See our [guide to serialization & saving](https://www.tensorflow.org/guide/keras/save_and_serialize/).
- Speed up model training by leveraging multiple GPUs. See our [guide to multi-GPU and distributed training](https://keras.io/guides/distributed_training/).

<br>
<br>
<br>

## Feature extraction with a Sequential model

<font size=3 color=maroon>Once a Sequential model has been built, it behaves like a [Functional API
model](https://www.tensorflow.org/guide/keras/functional/). This means that every layer has an `input` and `output` attribute.</font> These attributes can be used to do neat things, like quickly
creating a model that extracts the outputs of all intermediate layers in a Sequential model:

In [22]:
initial_model = keras.Sequential([keras.Input(shape=(250, 250, 3)),
                                  layers.Conv2D(32, 5, strides=2, activation="relu"),
                                  layers.Conv2D(32, 3, activation="relu"),
                                  layers.Conv2D(32, 3, activation="relu"),
                                 ])


feature_extractor = keras.Model(inputs=initial_model.inputs,
                                outputs=[layer.output for layer in initial_model.layers],
                               )

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

<br>

In [23]:
initial_model.inputs

[<KerasTensor: shape=(None, 250, 250, 3) dtype=float32 (created by layer 'input_3')>]

In [24]:
initial_model.layers

In [25]:
initial_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 123, 123, 32)      2432      
                                                                 
 conv2d_7 (Conv2D)           (None, 121, 121, 32)      9248      
                                                                 
 conv2d_8 (Conv2D)           (None, 119, 119, 32)      9248      
                                                                 
Total params: 20,928
Trainable params: 20,928
Non-trainable params: 0
_________________________________________________________________


<br>

In [26]:
print(type(features))
print(len(features))

<class 'list'>
3


In [27]:
for f in features:
    print(type(f))
    print(f.shape)
    print()

<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 123, 123, 32)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 121, 121, 32)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 119, 119, 32)



In [28]:
# features

<br>

Here's a similar example that only extract features from one layer:

In [29]:
initial_model = keras.Sequential([keras.Input(shape=(250, 250, 3)),
                                  layers.Conv2D(32, 5, strides=2, activation="relu"),
                                  layers.Conv2D(32, 3, activation="relu", 
                                                name="my_intermediate_layer"),
                                  layers.Conv2D(32, 3, activation="relu"),
                                 ])

feature_extractor = keras.Model(inputs=initial_model.inputs,
                                outputs=initial_model.get_layer(name="my_intermediate_layer")\
                                                     .output,
                               )


# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [30]:
print(type(features))
print(len(features))
print()

for f in features:
    print(type(f))
    print(f.shape)
    print()

<class 'tensorflow.python.framework.ops.EagerTensor'>
1

<class 'tensorflow.python.framework.ops.EagerTensor'>
(121, 121, 32)



<br>
<br>
<br>

## Transfer learning with a Sequential model

<font size=3 color=maroon>Transfer learning consists of freezing the bottom layers in a model and only training the top layers.</font> If you aren't familiar with it, make sure to read our [guide to transfer learning](https://www.tensorflow.org/guide/keras/transfer_learning/).

**Here are two common transfer learning blueprint involving Sequential models.**

<font size=3 color=maroon>First, let's say that you have a Sequential model, and you want to freeze all layers except the last one. In this case, you would simply iterate over `model.layers` and set `layer.trainable = False` on each layer, except the last one. Like this:</font>

```python
model = keras.Sequential([keras.Input(shape=(784)),
                          layers.Dense(32, activation='relu'),
                          layers.Dense(32, activation='relu'),
                          layers.Dense(32, activation='relu'),
                          layers.Dense(10),
                         ])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
    layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)
```

<br>

Another common blueprint is to use a Sequential model to stack a pre-trained model and some freshly initialized classification layers. Like this:

```python
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(weights='imagenet',
                                         include_top=False,
                                         pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([base_model,
                          layers.Dense(1000),
                         ])

# Compile & train
model.compile(...)
model.fit(...)
```

<br>

If you do transfer learning, you will probably find yourself frequently using these two patterns.

<br>
<br>
<br>

That's about all you need to know about Sequential models!

To find out more about building models in Keras, see:

- [Guide to the Functional API](https://www.tensorflow.org/guide/keras/functional/)
- [Guide to making new Layers & Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models/)

<br>
<br>
<br>